In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:

import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
from datasets import Dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
)
import torch


In [3]:

from google.colab import files
uploaded = files.upload()  # Upload the EdSheeran_Coldplay_Lyrics.xlsx file

df = pd.read_excel("EdSheeran_Coldplay_Lyrics.xlsx")
df = df.dropna(subset=["LYRICS"])
df = df[df["LYRICS"].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]
dataset = Dataset.from_dict({"text": df["LYRICS"].tolist()})


Saving EdSheeran_Coldplay_Lyrics.xlsx to EdSheeran_Coldplay_Lyrics (1).xlsx


In [4]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [6]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-ed-coldplay",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available()
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


<ipython-input-7-d748906289dd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=15, training_loss=3.4093396504720053, metrics={'train_runtime': 106.8941, 'train_samples_per_second': 0.281, 'train_steps_per_second': 0.14, 'total_flos': 1959690240000.0, 'train_loss': 3.4093396504720053, 'epoch': 3.0})

In [8]:

trainer.save_model("./gpt2-ed-coldplay")
tokenizer.save_pretrained("./gpt2-ed-coldplay")


('./gpt2-ed-coldplay/tokenizer_config.json',
 './gpt2-ed-coldplay/special_tokens_map.json',
 './gpt2-ed-coldplay/vocab.json',
 './gpt2-ed-coldplay/merges.txt',
 './gpt2-ed-coldplay/added_tokens.json')

In [9]:

generator = pipeline('text-generation', model="./gpt2-ed-coldplay", tokenizer=tokenizer)

prompts = [
    "I'm in love with the shape",
    "Look at the stars, look how",
]

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    output = generator(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print(f"Generated Lyrics:\n{output}")


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Prompt: I'm in love with the shape
Generated Lyrics:
I'm in love with the shape. And I love the feeling."

The young boy loves going to the gym.

While his mother didn't know what his parents wanted, they never told him anything he wasn't doing.

His mother was in a state of excitement about getting his body ready for the next steps. As a boy, she thought it was like a big leap.

"I was like, 'Oh my god, I can do this,'" she said

Prompt: Look at the stars, look how
Generated Lyrics:
Look at the stars, look how they shine now and then, that's like a mirror and you can see it from the deep sky because it is so beautiful. But now they're getting dim and they're getting dim by the minute, they don't give me a thing, I need other people. My husband can see all the stars. He is just a little bit bigger than what this world is, the stars are my soul and mine. I love them so much, I've never had
